In [1]:
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from tqdm import tqdm
from functions import MODEL

from collections import Counter
from prettytable import PrettyTable
from IPython.display import Image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Concatenate, Layer
from keras.layers import Conv1D, Add, MaxPooling1D, BatchNormalization
from keras.layers import Embedding, Bidirectional, GlobalMaxPooling1D, LSTM
import keras.backend as K

obj = MODEL()

In [2]:
# Import and process level 2 data
drug_smiles = pd.read_csv('data/smiles.csv')[['drug','smile']]  # Drug index dataframe
drug_smiles['seq_char_count'] = drug_smiles['smile'].apply(lambda x: len(str(x)))   # Add column with character count
drug_smiles = drug_smiles[drug_smiles['smile'].notnull()]   # Remove drugs with no smile strings
DTI_index = pd.read_csv('data/final_clean_DTI.csv')[['target','drug','IC50','unit']]
DTI_index = DTI_index[DTI_index['drug'].isin(drug_smiles['drug'].tolist())]     # Exclude drugs for which smils are not available
DTI_index = DTI_index.reset_index(drop=True)
mapping = pd.read_csv('data/chembl2uniprot.txt', header=None, sep='\t')
mapping = mapping[mapping[0].isin(DTI_index['target'].unique())]    # Select targets that are present in data (DTI_index)
targets = os.listdir('data/fasta_968')  # List all target fasta files

# Categorize drug-target pairs by IC50 values (Make labels)
act = []
bct = []
for i in tqdm(range(DTI_index.shape[0])):
    if DTI_index['IC50'][i]<=0.1:
        act.append('active')
    elif DTI_index['IC50'][i]>0.1 and DTI_index['IC50'][i]<=30:
        act.append('intermediate')
    elif DTI_index['IC50'][i]>30:
        act.append('inactive')
    bct.append(mapping[mapping[0]==DTI_index['target'][i]][1].values[0])
    
DTI_index['activity'] = act
DTI_index['target_uniprot'] = bct

100%|██████████| 76784/76784 [01:08<00:00, 1123.22it/s]


In [3]:
# Fetch fasta sequences form files and create target index dataframe
def fetchFasta(targets):
    target_seq = pd.DataFrame(columns=['target_uniprot','target_chembl','seq'])
    for fasta in tqdm(targets):
        if fasta.split('.')[0] in mapping[1].tolist():
            f = open('data/fasta_968/'+fasta,'r')
            lines = "".join(line.strip() for line in f.readlines()[1:])
            dict = {'target_uniprot':fasta.split('.')[0],'target_chembl':mapping[(mapping[1]==fasta.split('.')[0])][0].values[0], 'seq':lines}
            target_seq = target_seq.append(dict, True)
            f.close()
    return target_seq

target_seq = fetchFasta(targets)
# Length of sequence in train data.
#target_seq['seq_char_count'] = target_seq['seq'].apply(lambda : len(x))

100%|██████████| 968/968 [00:05<00:00, 180.63it/s]


In [5]:
# Split indics into train/test
train_indices, test_indices = obj.split(DTI_index, 8)

In [6]:
# Train and test data
train_target = DTI_index.loc[train_indices][['target_uniprot']]
train_drug = DTI_index.loc[train_indices][['drug']]
test_target = DTI_index.loc[test_indices][['target_uniprot']]
test_drug = DTI_index.loc[test_indices][['drug']]

# Labels
train_y = DTI_index.loc[train_indices][['activity']]
test_y = DTI_index.loc[test_indices][['activity']]

print(train_target.shape, train_drug.shape, test_target.shape, test_drug.shape, train_y.shape, test_y.shape)

(61427, 1) (61427, 1) (15357, 1) (15357, 1) (61427, 1) (15357, 1)


In [7]:
# Add uniprot IDs to data index
seq = []
for target in tqdm(train_target['target_uniprot']):
    try:
        seq.append(target_seq[target_seq['target_uniprot']==target]['seq'].values[0])
    except:
        print(target)
train_target['seq'] = seq

100%|██████████| 61427/61427 [00:49<00:00, 1243.45it/s]


In [8]:
# Add smile strings to labels
seq = []
for drug in tqdm(train_drug['drug']):
    try:
        seq.append(drug_smiles[drug_smiles['drug']==drug]['smile'].values[0])
    except:
        print(target)
train_drug['seq'] = seq

100%|██████████| 61427/61427 [03:59<00:00, 256.34it/s]


In [9]:

# plt.subplot(1, 1, 1)
# obj.plot_seq_count(drug_smiles, 'Train')

# code_freq = obj.get_code_freq(target_seq['seq'], 'Train')
# plt.subplot(1, 1, 1)
# obj.plot_code_freq(code_freq, 'Train')

In [10]:
# Encode amino acides and smile characters
codes_target = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
char_dict_target = obj.create_dict(codes_target)

codes_drug = [char for char in ''.join(set(''.join(drug_smiles['smile'].values)))]
char_dict_drug = obj.create_dict(codes_drug)

print(char_dict_target)
print("Target Dict Length:", len(char_dict_target))

print(char_dict_drug)
print("Drug dict Length:", len(char_dict_drug))

{'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
Target Dict Length: 20
{'3': 1, 'a': 2, 'n': 3, 'F': 4, '#': 5, 'l': 6, 'N': 7, 'o': 8, '.': 9, '7': 10, 'e': 11, '1': 12, '6': 13, '2': 14, 's': 15, '\\': 16, 'S': 17, '-': 18, '8': 19, 'Z': 20, 'r': 21, 'A': 22, '4': 23, '[': 24, 'c': 25, ')': 26, '@': 27, 'K': 28, 'i': 29, 'I': 30, '/': 31, '(': 32, '9': 33, '=': 34, '+': 35, 'H': 36, 'B': 37, 'L': 38, '5': 39, ']': 40, 'P': 41, 'C': 42, 'O': 43}
Drug dict Length: 43


In [11]:
train_encode_target = obj.integer_encoding(train_target, char_dict_target) 
train_encode_drug = obj.integer_encoding(train_drug, char_dict_drug) 

In [12]:
# padding sequences
max_length = 1000
train_pad_target = pad_sequences(train_encode_target, maxlen=max_length, padding='post', truncating='post')
train_pad_drug = pad_sequences(train_encode_drug, maxlen=max_length, padding='post', truncating='post')
train_pad_target.shape, train_pad_drug.shape

((61427, 1000), (61427, 1000))

In [13]:
# One hot encoding of sequences
train_ohe_target = to_categorical(train_pad_target)
train_ohe_drug = to_categorical(train_pad_drug)
train_ohe_target.shape, train_ohe_drug.shape

((61427, 1000, 21), (61427, 1000, 44))

In [13]:
# label/integer encoding output variable: (y)
le = LabelEncoder()
y_train_le = le.fit_transform(train_y['activity'].tolist())
y_train_le.shape

(61427,)

In [14]:
# One hot encoding of outputs
y_train = to_categorical(y_train_le)
y_train.shape

(61427, 3)

In [22]:
# Attention class
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [23]:
# Model Architecture
input_target = Input(shape=(1000,))
emb_target = Embedding(21, 128, input_length=max_length)(input_target) 
conv_target_1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(emb_target)
pool_target_1 = MaxPooling1D(pool_size=2)(conv_target_1)
att_in_target = Bidirectional(LSTM(32, kernel_regularizer=l2(0.01), return_sequences=True, recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))(pool_target_1)
att_out_target = attention()(att_in_target)
flat_1_target = Flatten()(att_out_target)

# softmax classifier
#x_output_target = Dense(3, activation='softmax')(att_in_target)

input_drug = Input(shape=(1000,))
emb_drug = Embedding(44, 128, input_length=max_length)(input_drug) 
conv_drug_1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(emb_drug)
pool_drug_1 = MaxPooling1D(pool_size=2)(conv_drug_1)
att_in_drug = Bidirectional(LSTM(32, kernel_regularizer=l2(0.01), return_sequences=True, recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))(pool_drug_1)
att_out_drug = attention()(att_in_drug)
flat_1_drug = Flatten()(att_out_drug)

concat = Concatenate()([flat_1_target,flat_1_drug])

dense_1 = Dense(256, activation = 'relu',kernel_initializer='glorot_normal')(concat)
dense_2 = Dense(128, activation = 'relu',kernel_initializer='glorot_normal')(dense_1)

# softmax classifier
x_output = Dense(3, activation='softmax')(dense_2)

model1 = Model(inputs=[input_target, input_drug], outputs=x_output)
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1000, 128)    2688        input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1000, 128)    5632        input_6[0][0]                    
_______________________________________________________________________________________

In [24]:
# Early Stopping
es = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
histories = []

In [26]:
histories.append(model1.fit(
    [train_pad_target, train_pad_drug], y_train,
    epochs=100, batch_size=128,
    validation_split=0.2,
    callbacks=[es]
    ))

Epoch 1/100
 55/384 [===>..........................] - ETA: 13:42 - loss: 3.4658 - accuracy: 0.4878

KeyboardInterrupt: 

In [ ]:
# Plot model history
obj.plot_history(histories[0])